In [2]:
!pip install neo4j

In [3]:
import mysql.connector
from neo4j import GraphDatabase

# MySQL connection
mysql_conn = mysql.connector.connect(
    host='127.0.0.1',
    user='root',
    password='root',
    database='eccubedb',
    port=13306
)

# Neo4j connection
neo4j_driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "ttt@123ASD"))

# Create a cursor object for MySQL
mysql_cursor = mysql_conn.cursor()

In [4]:
# Execute the query to get all tables in the schema
mysql_cursor.execute("SHOW TABLES")

# Fetch all tables
tables = mysql_cursor.fetchall()

# Initialize a dictionary to store the schema
schema = {}

# Iterate through each table
for table in tables:
    table_name = table[0]
    schema[table_name] = {
        "columns": [],
        "foreign_keys": []
    }
    
    # Execute the query to get all columns of the table
    mysql_cursor.execute(f"SHOW COLUMNS FROM {table_name}")
    columns = mysql_cursor.fetchall()
    
    # Add columns to the schema
    for column in columns:
        schema[table_name]["columns"].append(column[0])
    
    # Execute the query to get foreign keys of the table
    mysql_cursor.execute(f"""
        SELECT
            column_name,
            referenced_table_name,
            referenced_column_name
        FROM
            information_schema.key_column_usage
        WHERE
            table_name = '{table_name}' AND
            referenced_table_name IS NOT NULL
    """)
    foreign_keys = mysql_cursor.fetchall()
    
    # Add foreign keys to the schema
    for fk in foreign_keys:
        schema[table_name]["foreign_keys"].append({
            "column": fk[0],
            "referenced_table": fk[1],
            "referenced_column": fk[2]
        })

# Close the MySQL cursor and connection
mysql_cursor.close()
mysql_conn.close()



In [5]:

# Function to create nodes and relationships in Neo4j
def create_schema_in_neo4j(schema):
    with neo4j_driver.session() as session:
        # Create nodes for each table
        for table_name, details in schema.items():
            session.run(f"CREATE (t:Table {{name: '{table_name}'}})")
            
            # Create nodes for each column
            for column in details["columns"]:
                session.run(f"""
                    MATCH (t:Table {{name: '{table_name}'}})
                    CREATE (c:Column {{name: '{column}'}})-[:BELONGS_TO]->(t)
                """)
            
            # Create relationships for foreign keys
            for fk in details["foreign_keys"]:
                session.run(f"""
                    MATCH (t1:Table {{name: '{table_name}'}}),
                          (t2:Table {{name: '{fk['referenced_table']}'}})
                    CREATE (t1)-[:FOREIGN_KEY {{column: '{fk['column']}', referenced_column: '{fk['referenced_column']}'}}]->(t2)
                """)

# Create the schema in Neo4j
create_schema_in_neo4j(schema)

In [12]:
def get_all_tables_and_relationships(filter=None):
    query = """
    MATCH (n:Table)
    OPTIONAL MATCH (t1:Table)-[r:FOREIGN_KEY]->(t2:Table)
    RETURN n, t1, r, t2
    LIMIT 1000
    """

    with neo4j_driver.session() as session:
        result = session.run(query)
        for record in result:
            print('record', record)
            table_node = record['n']
            print('table_node', table_node, type(table_node))
            table_properties = table_node._properties
            print('table_properties', table_properties)
            if table_properties is not None:
                table_name = table_properties.get('name', 'Unknown')
                print(f"Table: {table_name}")
            print(f"Identity: {table_node['identity']}")
            print(f"Labels: {table_node['labels']}")
            print(f"ElementId: {table_node['elementId']}")
            print("Relationships:")
            # for relationship in record['relationships']:
            #     if relationship['from'] and relationship['to']:
            #         print(f"  - {relationship['from']} references {relationship['to']}({relationship['column']} -> {relationship['referenced_column']})")
            print()

# Get all tables and their relationships with an optional filter
get_all_tables_and_relationships('dcm_areas')

record <Record n=<Node element_id='4:c9175aa1-74a8-47eb-9110-710878eb2fec:276' labels=frozenset({'Table'}) properties={'name': 'dcm_areas'}> t1=<Node element_id='4:c9175aa1-74a8-47eb-9110-710878eb2fec:283' labels=frozenset({'Table'}) properties={'name': 'dcm_atms'}> r=<Relationship element_id='5:c9175aa1-74a8-47eb-9110-710878eb2fec:11261' nodes=(<Node element_id='4:c9175aa1-74a8-47eb-9110-710878eb2fec:283' labels=frozenset({'Table'}) properties={'name': 'dcm_atms'}>, <Node element_id='4:c9175aa1-74a8-47eb-9110-710878eb2fec:5110' labels=frozenset({'Table'}) properties={'name': 'dcm_shop'}>) type='FOREIGN_KEY' properties={'referenced_column': 'id', 'column': 'shop_id'}> t2=<Node element_id='4:c9175aa1-74a8-47eb-9110-710878eb2fec:5110' labels=frozenset({'Table'}) properties={'name': 'dcm_shop'}>>
table_node <Node element_id='4:c9175aa1-74a8-47eb-9110-710878eb2fec:276' labels=frozenset({'Table'}) properties={'name': 'dcm_areas'}> <class 'neo4j.graph.Node'>
table_properties {'name': 'dcm_ar